# LBM Step 4: Collate Planes

In [13]:
import sys
from pathlib import Path
import os
import numpy as np
import zarr
import napari

import logging
import matplotlib.pyplot as plt

# Importing this WONT WORK withotu adding to the path
sys.path.append('../..')  # TODO: Take this out when we upload to pypi
import lbm_caiman_python as lcp

import mesmerize_core as mc
from mesmerize_viz import *
from mesmerize_core.caiman_extensions.cnmf import cnmf_cache

try:
    import cv2
    cv2.setNumThreads(0)
except():
    pass

logging.basicConfig()

os.environ["CONDA_PREFIX_1"] = ""
if os.name == "nt":
    # disable the cache on windows, this will be automatic in a future version
    cnmf_cache.set_maxsize(0)


In [14]:
parent_path = Path().home() / "caiman_data" / 'animal_01' / "session_01"

batch_path = parent_path / 'batch.pickle'
mc.set_parent_raw_data_path(str(parent_path))

# create a new batch
try:
    df = mc.load_batch(batch_path)
except (IsADirectoryError, FileNotFoundError):
    df = mc.create_batch(batch_path)

df=df.caiman.reload_from_disk()
df

,algo,item_name,input_movie_path,params,outputs,added_time,ran_time,algo_duration,comments,uuid
0,mcorr,extracted_plane_1,tiff\extracted_plane_1.tif,"{'main': {'var_name_hdf5': 'mov', 'max_shifts'...",{'mean-projection-path': b32f41bf-a9a5-4965-be...,2024-09-26T11:56:54,2024-09-26T12:02:55,77.3 sec,None,b32f41bf-a9a5-4965-be7c-e6779e854328
1,cnmf,cnmf_1,b32f41bf-a9a5-4965-be7c-e6779e854328\b32f41bf-...,"{'main': {'fr': 9.62, 'dxy': (1.0, 1.0), 'deca...",{'mean-projection-path': a057e39e-a2df-41d3-82...,2024-09-26T16:26:20,2024-09-26T16:28:48,143.18 sec,None,a057e39e-a2df-41d3-8217-83c9cd7ffb6d
2,cnmf,extracted_plane_1,b32f41bf-a9a5-4965-be7c-e6779e854328\b32f41bf-...,"{'main': {'fr': 9.62, 'dxy': (1.0, 1.0), 'deca...",{'mean-projection-path': 5cb543ec-5358-4b35-83...,2024-09-27T14:48:59,2024-09-27T14:50:30,80.14 sec,None,5cb543ec-5358-4b35-83cf-bfd19fa06a68
3,cnmf,extracted_plane_1,b32f41bf-a9a5-4965-be7c-e6779e854328\b32f41bf-...,"{'main': {'fr': 9.62, 'dxy': (1.0, 1.0), 'deca...",{'mean-projection-path': df52a609-d986-4bc0-a0...,2024-09-27T14:48:59,2024-09-27T14:51:46,75.93 sec,None,df52a609-d986-4bc0-a0a2-526cbfda057d
4,cnmf,extracted_plane_1,b32f41bf-a9a5-4965-be7c-e6779e854328\b32f41bf-...,"{'main': {'fr': 9.62, 'dxy': (1.0, 1.0), 'deca...",{'mean-projection-path': b68a8efe-1a3b-4392-b6...,2024-09-27T14:48:59,2024-09-30T10:30:15,96.62 sec,None,b68a8efe-1a3b-4392-b6e7-416312f1f945
5,cnmf,extracted_plane_1,b32f41bf-a9a5-4965-be7c-e6779e854328\b32f41bf-...,"{'main': {'fr': 9.62, 'dxy': (1.0, 1.0), 'deca...",{'mean-projection-path': 15e6fb81-fac4-457b-96...,2024-09-27T14:48:59,2024-09-30T11:32:09,89.61 sec,None,15e6fb81-fac4-457b-96da-dbb8e2d7b669
6,cnmf,k=40,b32f41bf-a9a5-4965-be7c-e6779e854328\b32f41bf-...,"{'main': {'fr': 9.62, 'dxy': (1.0, 1.0), 'deca...",{'mean-projection-path': abba269d-f2f3-467b-ba...,2024-09-30T14:02:10,2024-09-30T14:05:00,165.71 sec,None,abba269d-f2f3-467b-ba40-f1e33aea8461
7,cnmf,cnmf_1,b32f41bf-a9a5-4965-be7c-e6779e854328\b32f41bf-...,"{'main': {'fr': 9.62, 'dxy': (1.0, 1.0), 'deca...",{'mean-projection-path': 9777376d-092c-43db-b1...,2024-09-30T14:36:58,2024-09-30T14:41:40,161.74 sec,None,9777376d-092c-43db-b14a-c9be1e142e5c
8,cnmf,cnmf_1,b32f41bf-a9a5-4965-be7c-e6779e854328\b32f41bf-...,"{'main': {'fr': 9.62, 'dxy': (1.0, 1.0), 'deca...",{'mean-projection-path': 88177367-8fbf-499b-94...,2024-09-30T14:36:58,2024-09-30T14:44:32,172.71 sec,None,88177367-8fbf-499b-940e-1fb9c331b499
9,cnmf,cnmf_1,b32f41bf-a9a5-4965-be7c-e6779e854328\b32f41bf-...,"{'main': {'fr': 9.62, 'dxy': (1.0, 1.0), 'deca...",{'mean-projection-path': a3bd82e1-618e-4279-8a...,2024-09-30T14:36:58,2024-09-30T14:47:31,179.02 sec,None,a3bd82e1-618e-4279-8a67-ae3fe52a10d2


In [7]:
results = {}
for i, row in df.iterrows():
    if row["outputs"] is not None and row['algo']=='cnmf': # item has already been run
        results[i] = df.iloc[i].cnmf.get_output()

results

{1: <caiman.source_extraction.cnmf.cnmf.CNMF at 0x1efebf66500>,
 2: <caiman.source_extraction.cnmf.cnmf.CNMF at 0x1ef80fdd210>,
 3: <caiman.source_extraction.cnmf.cnmf.CNMF at 0x1ef80fdcd90>,
 4: <caiman.source_extraction.cnmf.cnmf.CNMF at 0x1ef80fdc610>,
 5: <caiman.source_extraction.cnmf.cnmf.CNMF at 0x1ef80fdcf40>,
 6: <caiman.source_extraction.cnmf.cnmf.CNMF at 0x1ef80fdc6a0>,
 7: <caiman.source_extraction.cnmf.cnmf.CNMF at 0x1ef80f69480>,
 8: <caiman.source_extraction.cnmf.cnmf.CNMF at 0x1ef80fdf820>,
 9: <caiman.source_extraction.cnmf.cnmf.CNMF at 0x1ef80fdf130>,
 10: <caiman.source_extraction.cnmf.cnmf.CNMF at 0x1ef80fdd8a0>,
 11: <caiman.source_extraction.cnmf.cnmf.CNMF at 0x1ef80fdc4c0>,
 12: <caiman.source_extraction.cnmf.cnmf.CNMF at 0x1ef80fdf580>,
 13: <caiman.source_extraction.cnmf.cnmf.CNMF at 0x1ef80f77130>,
 14: <caiman.source_extraction.cnmf.cnmf.CNMF at 0x1ef80f77850>,
 15: <caiman.source_extraction.cnmf.cnmf.CNMF at 0x1ef80f767d0>,
 16: <caiman.source_extraction.cnm

In [16]:
from ipywidgets import Tab, interact_manual
@interact_manual(parent_path=str(parent_path), batch_path=str(batch_path))
def start_widget(parent_path, batch_path):
    mc.set_parent_raw_data_path(parent_path)

    df = mc.load_batch(batch_path)

    tab = Tab()

    # mcorr_container = df.mcorr.viz()
    cnmf_container = df.cnmf.viz(start_index=3)

    tab.children = [cnmf_container.show()]
    tab.titles = ["cnmf"]
    display(tab)

interactive(children=(Text(value='C:\\Users\\RBO\\caiman_data\\animal_01\\session_01', continuous_update=False…

In [20]:
import fastplotlib as fpl
# get the reconstructed background as a LazyArray
# assumes the last index, `-1`, is a cnmf item
rcb = df.iloc[-1].cnmf.get_rcb()

# view with ImageWidget
iw = fpl.ImageWidget(data=rcb, show_toolbar=False)
iw.show()

RFBOutputContext()

TypeError: Graphic.__init__() got an unexpected keyword argument 'show_toolbar'

In [ ]:
widget = fpl.ImageWidget()